# Stencil Numba MPI

In [1]:
! python --version

Python 3.7.3


In [2]:
import numba
print(numba.__version__)

0.41.0


In [10]:
# Mostra os recursos do nó de login
! lscpu | head -n 15 | grep "Model \|CPU(s):\|Thre\|Core\|NUMA\|MHz"

CPU(s):                24
Thread(s) per core:    1
Core(s) per socket:    12
NUMA node(s):          2
Model name:            Intel(R) Xeon(R) CPU E5-2695 v2 @ 2.40GHz
CPU MHz:               2849.707


# Numba CPU Paralelo
24 núcleos
Testando o funcionamento no nó de login, antes de usar o arquivo de lote

In [26]:
import numpy as np
import time
from mpi4py import MPI   

from numba import jit, prange, config
config.DUMP_ASSEMBLY = 0
config.NUMBA_ENABLE_AVX = 1
config.NUMBA_NUM_THREADS = 1

@jit('(float64[:,:],float64[:,:])', nopython=True, parallel=True, nogil=True) 
def kernel1(anew, aold) :
    anew[1:-1, 1:-1] = ( aold[1:-1, 1:-1] * 0.5 + 
                       ( aold[2:  , 1:-1] + aold[ :-2, 1:-1] +
                         aold[1:-1, 2:  ] + aold[1:-1,  :-2] ) * 0.125 )


n            = 4800    # nxn grid (4800,1,500)=1500; (100,1,10)=30
energy       = 1.0     # energy to be injected per iteration
niters       = 500     # number of iterations

nsources     = 3       # sources of energy
size         = n + 2
heat         = np.zeros((1), np.float64)     # system total heat
anew         = np.zeros((size, size), np.float64)
aold         = np.zeros((size, size), np.float64)
sources      = np.empty((3,2), np.int32)
sources[:,:] = [ [n//2, n//2], [n//3, n//3], [n*4//5, n*8//9] ]
niters       = (niters+1) // 2

comm = MPI.COMM_WORLD
mpirank = comm.rank
mpisize = comm.size

# cria e inicializa as fontes de calor
nsources = 3
sources = np.zeros((nsources, 2), np.intc)
sources[:,:] = [ [n//2, n//2], [n//3, n//3], [n*4//5, n*8//9] ]

# sources in my area, local to my rank
locnsources = 0
locsources = np.empty((nsources,2), np.intc)

rheat = np.zeros(1, np.double)
bheat = np.zeros(1, np.double)

# determine my coordinates (x,y)
pdims = MPI.Compute_dims(mpisize, 2)
px    = pdims[0]
py    = pdims[1]
rx    = mpirank % px
ry    = mpirank // px

# determine my four neighbors
north = (ry - 1) * px + rx
if (ry - 1) < 0 :
    north = MPI.PROC_NULL
south = (ry + 1) * px + rx
if (ry + 1) >= py :
    south = MPI.PROC_NULL
west = ry * px + rx - 1
if (rx - 1) < 0 :
    west = MPI.PROC_NULL
east = ry * px + rx + 1
if (rx + 1) >= px :
    east = MPI.PROC_NULL

# decompose the domain
bx = n // px            # block size in x
by = n // py            # block size in y
offx = rx * bx + 1      # offset in x
offy = ry * by + 1      # offset in y

# determine which sources are in my patch
for i in range(nsources) :
    locx = sources[i, 0] - offx
    locy = sources[i, 1] - offy
    if(locx >= 0 and locx <= bx and locy >= 0 and locy <= by) :
        locsources[locnsources, 0] = locx + 2 - 1
        locsources[locnsources, 1] = locy + 2 - 1
        locnsources += 1

# working arrays with 1-wide halo zones
anew = np.zeros((bx+2, by+2), np.double)
aold = np.zeros((bx+2, by+2), np.double)

if not mpirank : t0 = time.time()

for iters in range(niters) :
    # exchange data with neighbors
    if north != MPI.PROC_NULL :
        r1=comm.irecv(source=north, tag=1)
        s1=comm.isend(aold[1, 1:bx+1], dest=north, tag=1)
    if south != MPI.PROC_NULL :
        r2=comm.irecv(source=south, tag=1)
        s2=comm.isend(aold[bx, 1:bx+1], dest=south, tag=1)
    if east != MPI.PROC_NULL :
        r3 = comm.irecv(source=east, tag=1)
        s3 = comm.isend(aold[1:bx+1, bx], dest=east, tag=1)
    if west != MPI.PROC_NULL :
        r4 = comm.irecv(source=west, tag=1)
        s4 = comm.isend(aold[1:bx+1, 1], dest=west, tag=1)
    # wait
    if north != MPI.PROC_NULL :
        s1.wait()
        aold[0, 1:bx+1] = r1.wait()
    if south != MPI.PROC_NULL :
        s2.wait()
        aold[bx+1, 1:bx+1] = r2.wait()
    if east != MPI.PROC_NULL :
        s3.wait()
        aold[1:bx+1, bx+1] = r3.wait()
    if west != MPI.PROC_NULL :
        s4.wait
        aold[1:bx+1, 0] = r4.wait()

    # update grid
    kernel1(anew, aold)

    # refresh heat sources
    for i in range(locnsources) :
        anew[locsources[i, 0]-1, locsources[i, 1]-1] += energy

    # exchange data with neighbors
    if north != MPI.PROC_NULL :
        r1=comm.irecv(source=north, tag=1)
        s1=comm.isend(anew[1, 1:bx+1], dest=north, tag=1)
    if south != MPI.PROC_NULL :
        r2=comm.irecv(source=south, tag=1)
        s2=comm.isend(anew[bx, 1:bx+1], dest=south, tag=1)
    if east != MPI.PROC_NULL :
        r3 = comm.irecv(source=east, tag=1)
        s3 = comm.isend(anew[1:bx+1, bx], dest=east, tag=1)
    if west != MPI.PROC_NULL :
        r4 = comm.irecv(source=west, tag=1)
        s4 = comm.isend(anew[1:bx+1, 1], dest=west, tag=1)
    # wait
    if north != MPI.PROC_NULL :
        s1.wait()
        anew[0, 1:bx+1] = r1.wait()
    if south != MPI.PROC_NULL :
        s2.wait()
        anew[bx+1, 1:bx+1] = r2.wait()
    if east != MPI.PROC_NULL :
        s3.wait()
        anew[1:bx+1, bx+1] = r3.wait()
    if west != MPI.PROC_NULL :
        s4.wait
        anew[1:bx+1, 0] = r4.wait()

    # update grid
    kernel1(aold, anew)

    # refresh heat sources
    for i in range(locnsources) :
        aold[locsources[i, 0]-1, locsources[i, 1]-1] += energy 

# get final heat in the system
bheat[0] = np.sum(aold[1:-1, 1:-1])
comm.Reduce(bheat, rheat)

if not mpirank :
    t0 = time.time() - t0
    print("Heat = %0.4f | Tempo = %0.4f | Thread count = %d"
          %(rheat[0], t0, config.NUMBA_NUM_THREADS))

Heat = 1500.0000 | Tempo = 22.1503 | Thread count = 1


# Rodando em um nó de execução
Grava o arquivo fonte:

In [3]:
%%writefile st-nu-par.py
import numpy as np
import time
from mpi4py import MPI   

from numba import jit, prange, config
config.DUMP_ASSEMBLY = 0
config.NUMBA_ENABLE_AVX = 1
config.NUMBA_NUM_THREADS = 1

@jit('(float64[:,:],float64[:,:])', nopython=True, parallel=True, nogil=True) 
def kernel1(anew, aold) :
    anew[1:-1, 1:-1] = ( aold[1:-1, 1:-1] * 0.5 + 
                       ( aold[2:  , 1:-1] + aold[ :-2, 1:-1] +
                         aold[1:-1, 2:  ] + aold[1:-1,  :-2] ) * 0.125 )


n            = 4800    # nxn grid (4800,1,500)=1500; (100,1,10)=30
energy       = 1.0     # energy to be injected per iteration
niters       = 500     # number of iterations

nsources     = 3       # sources of energy
size         = n + 2
heat         = np.zeros((1), np.float64)     # system total heat
anew         = np.zeros((size, size), np.float64)
aold         = np.zeros((size, size), np.float64)
sources      = np.empty((3,2), np.int32)
sources[:,:] = [ [n//2, n//2], [n//3, n//3], [n*4//5, n*8//9] ]
niters       = (niters+1) // 2

comm = MPI.COMM_WORLD
mpirank = comm.rank
mpisize = comm.size

# cria e inicializa as fontes de calor
nsources = 3
sources = np.zeros((nsources, 2), np.intc)
sources[:,:] = [ [n//2, n//2], [n//3, n//3], [n*4//5, n*8//9] ]

# sources in my area, local to my rank
locnsources = 0
locsources = np.empty((nsources,2), np.intc)

rheat = np.zeros(1, np.double)
bheat = np.zeros(1, np.double)

# determine my coordinates (x,y)
pdims = MPI.Compute_dims(mpisize, 2)
px    = pdims[0]
py    = pdims[1]
rx    = mpirank % px
ry    = mpirank // px

# determine my four neighbors
north = (ry - 1) * px + rx
if (ry - 1) < 0 :
    north = MPI.PROC_NULL
south = (ry + 1) * px + rx
if (ry + 1) >= py :
    south = MPI.PROC_NULL
west = ry * px + rx - 1
if (rx - 1) < 0 :
    west = MPI.PROC_NULL
east = ry * px + rx + 1
if (rx + 1) >= px :
    east = MPI.PROC_NULL

# decompose the domain
bx = n // px            # block size in x
by = n // py            # block size in y
offx = rx * bx + 1      # offset in x
offy = ry * by + 1      # offset in y

# determine which sources are in my patch
for i in range(nsources) :
    locx = sources[i, 0] - offx
    locy = sources[i, 1] - offy
    if(locx >= 0 and locx <= bx and locy >= 0 and locy <= by) :
        locsources[locnsources, 0] = locx + 2 - 1
        locsources[locnsources, 1] = locy + 2 - 1
        locnsources += 1

# working arrays with 1-wide halo zones
anew = np.zeros((bx+2, by+2), np.double)
aold = np.zeros((bx+2, by+2), np.double)

if not mpirank : t0 = time.time()

for iters in range(niters) :
    # exchange data with neighbors
    if north != MPI.PROC_NULL :
        r1=comm.irecv(source=north, tag=1)
        s1=comm.isend(aold[1, 1:bx+1], dest=north, tag=1)
    if south != MPI.PROC_NULL :
        r2=comm.irecv(source=south, tag=1)
        s2=comm.isend(aold[bx, 1:bx+1], dest=south, tag=1)
    if east != MPI.PROC_NULL :
        r3 = comm.irecv(source=east, tag=1)
        s3 = comm.isend(aold[1:bx+1, bx], dest=east, tag=1)
    if west != MPI.PROC_NULL :
        r4 = comm.irecv(source=west, tag=1)
        s4 = comm.isend(aold[1:bx+1, 1], dest=west, tag=1)
    # wait
    if north != MPI.PROC_NULL :
        s1.wait()
        aold[0, 1:bx+1] = r1.wait()
    if south != MPI.PROC_NULL :
        s2.wait()
        aold[bx+1, 1:bx+1] = r2.wait()
    if east != MPI.PROC_NULL :
        s3.wait()
        aold[1:bx+1, bx+1] = r3.wait()
    if west != MPI.PROC_NULL :
        s4.wait
        aold[1:bx+1, 0] = r4.wait()

    # update grid
    kernel1(anew, aold)

    # refresh heat sources
    for i in range(locnsources) :
        anew[locsources[i, 0]-1, locsources[i, 1]-1] += energy

    # exchange data with neighbors
    if north != MPI.PROC_NULL :
        r1=comm.irecv(source=north, tag=1)
        s1=comm.isend(anew[1, 1:bx+1], dest=north, tag=1)
    if south != MPI.PROC_NULL :
        r2=comm.irecv(source=south, tag=1)
        s2=comm.isend(anew[bx, 1:bx+1], dest=south, tag=1)
    if east != MPI.PROC_NULL :
        r3 = comm.irecv(source=east, tag=1)
        s3 = comm.isend(anew[1:bx+1, bx], dest=east, tag=1)
    if west != MPI.PROC_NULL :
        r4 = comm.irecv(source=west, tag=1)
        s4 = comm.isend(anew[1:bx+1, 1], dest=west, tag=1)
    # wait
    if north != MPI.PROC_NULL :
        s1.wait()
        anew[0, 1:bx+1] = r1.wait()
    if south != MPI.PROC_NULL :
        s2.wait()
        anew[bx+1, 1:bx+1] = r2.wait()
    if east != MPI.PROC_NULL :
        s3.wait()
        anew[1:bx+1, bx+1] = r3.wait()
    if west != MPI.PROC_NULL :
        s4.wait
        anew[1:bx+1, 0] = r4.wait()

    # update grid
    kernel1(aold, anew)

    # refresh heat sources
    for i in range(locnsources) :
        aold[locsources[i, 0]-1, locsources[i, 1]-1] += energy 

# get final heat in the system
bheat[0] = np.sum(aold[1:-1, 1:-1])
comm.Reduce(bheat, rheat)

if not mpirank :
    t1 = MPI.Wtime() - t0
    print('Heat={:0.4f} | Tempo={:0.4f} | MPISize={:d} | Dim={:d},{:d} | bx,by={:d},{:d}'
          .format(rheat[0], t1, mpisize, pdims[0], pdims[1], bx, by))

Overwriting st-nu-par.py


### Testa a execução

In [2]:
%%bash
mpiexec -n 1 python st-nu-par.py

Heat = 1500.0000 | Tempo = 22.1751 | Thread count = 1


In [17]:
%%bash
mpiexec -n 4 python st-nu-par.py

Heat=1500.0000 | Tempo=26.5325 | MPISize=4 | Dim=2,2 | bx,by=2400,2400


In [18]:
%%bash
mpiexec -n 9 python st-nu-par.py

Heat=1500.0000 | Tempo=3.8493 | MPISize=9 | Dim=3,3 | bx,by=1600,1600


### Copia para scratch

In [1]:
%%bash
a='/ampemi/xxxx.xxxx/stnc/Numba'
s='/prj'$a
d='/scratch'$a
cp  $s/st-nu-par.py  $d

In [13]:
%%writefile st-nu-par_81.srm
#!/bin/bash
# limites das filas (1,0 UA):
#   cpu_dev  : 20 min.,  1-4  nós, 1/1   tarefas em exec/fila máximo
#   cpu_small: 72 horas, 1-20 nós, 16/96 tarefas em exec/fila máximo
# 1x1=1, 2x2=4, 3x3=9, 4x4=16, 6x6=36, 7x7=49, 8x8=64, 9x9=81

#SBATCH --ntasks=81            #Total de tarefas
#SBATCH -J stnupar             #Nome do job, 8 caracteres
#SBATCH -p cpu_small           #Fila (partition) a ser utilizada
#SBATCH --time=00:02:00        #Tempo max. de execução 5 minutos
#SBATCH --exclusive            #Utilização exclusiva dos nós
# #SBATCH --nodes=1              #Qtd de nós
# #SBATCH --ntasks-per-node=1    #Qtd de tarefas por nó ($SLURM_NTASKS_PER_NODE)

echo '========================================'
echo '- Job ID:' $SLURM_JOB_ID
echo '- Tarefas por no:' $SLURM_NTASKS_PER_NODE
echo '- Qtd. de nos:' $SLURM_JOB_NUM_NODES
echo '- Tot. de tarefas:' $SLURM_NTASKS
echo '- Nos alocados:' $SLURM_JOB_NODELIST
nodeset -e $SLURM_JOB_NODELIST
echo '- diretorio onde sbatch foi chamado ($SLURM_SUBMIT_DIR):'
echo $SLURM_SUBMIT_DIR
cd $SLURM_SUBMIT_DIR

#Entra no diretório de trabalho
d='/scratch/ampemi/xxxx.xxxx/stnc/Numba'
cd $d

#Modulos
#module load mpi/openmpi3-x86_64
module load anaconda3/2018.12

#Executavel
EXEC='python st-nu-par.py'

#Dispara a execucao
#=======================================#
# ATENÇÃO! precisa do "--mpi=pmi2"      #
#=======================================#
echo '-- srun -------------------------------'
echo '$ srun --mpi=pmi2 -n' $SLURM_NTASKS $EXEC
srun --mpi=pmi2 -n $SLURM_NTASKS $EXEC
echo '-- FIM --------------------------------'

Overwriting st-nu-par_81.srm


### Testa a execução

In [45]:
%%bash
d='/scratch/ampemi/xxxx.xxxx/stnc/Numba'
cd $d
mpiexec -n 1 python st-nu-par.py

Heat=1500.0000 | Tempo=21.8506 | MPISize=1 | Dim=1,1 | bx,by=4800,4800


In [31]:
%%bash
d='/scratch/ampemi/xxxx.xxxx/stnc/Numba'
cd $d
mpiexec -n 4 python st-nu-par.py

Heat=1500.0000 | Tempo=27.0193 | MPISize=4 | Dim=2,2 | bx,by=2400,2400


In [47]:
%%bash
d='/scratch/ampemi/xxxx.xxxx/stnc/Numba'
cd $d
mpiexec -n 9 python st-nu-par.py

Heat=1500.0000 | Tempo=3.7629 | MPISize=9 | Dim=3,3 | bx,by=1600,1600


<hr style="height:10px;border-width:0;background-color:green">

### Envia para execução

In [4]:
%%bash
sbatch st-nu-par_04.srm

Submitted batch job 777730


In [5]:
! squeue -n stnupar

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)


In [6]:
%%bash
d='/scratch/ampemi/xxxx.xxxx/stnc/Numba'
cat $d/slurm-777730.out

- Job ID: 777730
- Tarefas por no:
- Qtd. de nos: 1
- Tot. de tarefas: 4
- Nos alocados: sdumont1099
sdumont1099
- diretorio onde sbatch foi chamado ($SLURM_SUBMIT_DIR):
/prj/ampemi/xxxx.xxxx/stnc/Numba
-- srun -------------------------------
$ srun --mpi=pmi2 -n 4 python st-nu-par.py
Heat=1500.0000 | Tempo=7.6107 | MPISize=4 | Dim=2,2 | bx,by=2400,2400
-- FIM --------------------------------


In [16]:
%%bash
# 1x1=1, 2x2=4, 3x3=9, 4x4=16, 6x6=36, 7x7=49, 8x8=64, 9x9=81
sbatch st-nu-par_01.srm
sbatch st-nu-par_09.srm
sbatch st-nu-par_16.srm
sbatch st-nu-par_36.srm
sbatch st-nu-par_49.srm
sbatch st-nu-par_64.srm
sbatch st-nu-par_81.srm

Submitted batch job 777755
Submitted batch job 777757
Submitted batch job 777758
Submitted batch job 777759
Submitted batch job 777760
Submitted batch job 777761
Submitted batch job 777762


In [18]:
! squeue -n stnupar

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)


In [19]:
%%bash
d='/scratch/ampemi/xxxx.xxxx/stnc/Numba'
cat $d/slurm-777755.out  #01
cat $d/slurm-777757.out  #09
cat $d/slurm-777758.out  #16
cat $d/slurm-777759.out  #36
cat $d/slurm-777760.out  #49
cat $d/slurm-777761.out  #64
cat $d/slurm-777762.out  #81

- Job ID: 777755
- Tarefas por no:
- Qtd. de nos: 1
- Tot. de tarefas: 1
- Nos alocados: sdumont1084
sdumont1084
- diretorio onde sbatch foi chamado ($SLURM_SUBMIT_DIR):
/prj/ampemi/xxxx.xxxx/stnc/Numba
-- srun -------------------------------
$ srun --mpi=pmi2 -n 1 python st-nu-par.py
Heat=1500.0000 | Tempo=30.4845 | MPISize=1 | Dim=1,1 | bx,by=4800,4800
-- FIM --------------------------------
- Job ID: 777757
- Tarefas por no:
- Qtd. de nos: 1
- Tot. de tarefas: 9
- Nos alocados: sdumont1099
sdumont1099
- diretorio onde sbatch foi chamado ($SLURM_SUBMIT_DIR):
/prj/ampemi/xxxx.xxxx/stnc/Numba
-- srun -------------------------------
$ srun --mpi=pmi2 -n 9 python st-nu-par.py
Heat=1500.0000 | Tempo=6.3707 | MPISize=9 | Dim=3,3 | bx,by=1600,1600
-- FIM --------------------------------
- Job ID: 777758
- Tarefas por no:
- Qtd. de nos: 1
- Tot. de tarefas: 16
- Nos alocados: sdumont1261
sdumont1261
- diretorio onde sbatch foi chamado ($SLURM_SUBMIT_DIR):
/prj/ampemi/xxxx.xxxx/stnc/Numba
-- 

In [1]:
! ls st-nu-par_*.srm

st-nu-par_01.srm  st-nu-par_09.srm  st-nu-par_36.srm  st-nu-par_64.srm
st-nu-par_04.srm  st-nu-par_16.srm  st-nu-par_49.srm  st-nu-par_81.srm


In [2]:
%%bash
# 1x1=1, 2x2=4, 3x3=9, 4x4=16, 6x6=36, 7x7=49, 8x8=64, 9x9=81
sbatch st-nu-par_04.srm

Submitted batch job 778875


In [4]:
! squeue -n stnupar

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
            778875 cpu_small  stnupar xxxx. PD       0:00      1 (Priority)


In [1]:
%%bash
d='/scratch/ampemi/xxxx.xxxx/stnc/Numba'
cat $d/slurm-778875.out

- Job ID: 778875
- Tarefas por no:
- Qtd. de nos: 1
- Tot. de tarefas: 4
- Nos alocados: sdumont1128
sdumont1128
- diretorio onde sbatch foi chamado ($SLURM_SUBMIT_DIR):
/prj/ampemi/xxxx.xxxx/stnc/Numba
-- srun -------------------------------
$ srun --mpi=pmi2 -n 4 python st-nu-par.py
Heat=1500.0000 | Tempo=7.5322 | MPISize=4 | Dim=2,2 | bx,by=2400,2400
-- FIM --------------------------------


<hr style="height:10px;border-width:0;background-color:green">

# Segunda tomada de tempo

In [1]:
! squeue -u xxxx.xxxx | grep -c ^    # limite 96

65


In [3]:
%%bash
# 1x1=1, 2x2=4, 3x3=9, 4x4=16, 6x6=36, 7x7=49, 8x8=64, 9x9=81
sbatch st-nu-par_01.srm
sbatch st-nu-par_04.srm
sbatch st-nu-par_09.srm
sbatch st-nu-par_16.srm
sbatch st-nu-par_36.srm
sbatch st-nu-par_49.srm
sbatch st-nu-par_64.srm
sbatch st-nu-par_81.srm

Submitted batch job 788106
Submitted batch job 788107
Submitted batch job 788108
Submitted batch job 788109
Submitted batch job 788110
Submitted batch job 788111
Submitted batch job 788112


In [7]:
%%bash
# 1x1=1, 2x2=4, 3x3=9, 4x4=16, 6x6=36, 7x7=49, 8x8=64, 9x9=81
#sbatch st-nu-par_01.srm
sbatch st-nu-par_04.srm
#sbatch st-nu-par_09.srm
#sbatch st-nu-par_16.srm
#sbatch st-nu-par_36.srm
#sbatch st-nu-par_49.srm
#sbatch st-nu-par_64.srm
#sbatch st-nu-par_81.srm

Submitted batch job 788313


In [4]:
! squeue -n stnupar

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
            788106 cpu_small  stnupar xxxx. PD       0:00      1 (Resources)
            788107 cpu_small  stnupar xxxx. PD       0:00      1 (Resources)
            788108 cpu_small  stnupar xxxx. PD       0:00      1 (Resources)
            788109 cpu_small  stnupar xxxx. PD       0:00      2 (Resources)
            788110 cpu_small  stnupar xxxx. PD       0:00      3 (Resources)
            788111 cpu_small  stnupar xxxx. PD       0:00      3 (Resources)
            788112 cpu_small  stnupar xxxx. PD       0:00      4 (Resources)


In [8]:
! squeue -n stnupar

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
            788313 cpu_small  stnupar xxxx. PD       0:00      1 (Priority)


In [13]:
%%bash
d='/scratch/ampemi/xxxx.xxxx/stnc/Numba'
cat $d/slurm-788106.out  #01
cat $d/slurm-788313.out  #04
cat $d/slurm-788107.out  #09
cat $d/slurm-788108.out  #16
cat $d/slurm-788109.out  #36
cat $d/slurm-788110.out  #49
cat $d/slurm-788111.out  #64
cat $d/slurm-788112.out  #81

- Job ID: 788106
- Tarefas por no:
- Qtd. de nos: 1
- Tot. de tarefas: 1
- Nos alocados: sdumont1149
sdumont1149
- diretorio onde sbatch foi chamado ($SLURM_SUBMIT_DIR):
/prj/ampemi/xxxx.xxxx/stnc/Numba
-- srun -------------------------------
$ srun --mpi=pmi2 -n 1 python st-nu-par.py
Heat=1500.0000 | Tempo=30.5498 | MPISize=1 | Dim=1,1 | bx,by=4800,4800
-- FIM --------------------------------
- Job ID: 788313
- Tarefas por no:
- Qtd. de nos: 1
- Tot. de tarefas: 4
- Nos alocados: sdumont1083
sdumont1083
- diretorio onde sbatch foi chamado ($SLURM_SUBMIT_DIR):
/prj/ampemi/xxxx.xxxx/stnc/Numba
-- srun -------------------------------
$ srun --mpi=pmi2 -n 4 python st-nu-par.py
Heat=1500.0000 | Tempo=8.4644 | MPISize=4 | Dim=2,2 | bx,by=2400,2400
-- FIM --------------------------------
- Job ID: 788107
- Tarefas por no:
- Qtd. de nos: 1
- Tot. de tarefas: 9
- Nos alocados: sdumont1149
sdumont1149
- diretorio onde sbatch foi chamado ($SLURM_SUBMIT_DIR):
/prj/ampemi/xxxx.xxxx/stnc/Numba
-- s

<hr style="height:10px;border-width:0;background-color:green">

# Terceira tomada de tempo

In [8]:
! squeue -u ${USER} | grep -c ^    # limite 96

79


In [6]:
%%bash
# 1x1=1, 2x2=4, 3x3=9, 4x4=16, 6x6=36, 7x7=49, 8x8=64, 9x9=81
sbatch st-nu-par_01.srm
sbatch st-nu-par_09.srm
sbatch st-nu-par_16.srm
sbatch st-nu-par_36.srm
sbatch st-nu-par_49.srm
sbatch st-nu-par_64.srm
sbatch st-nu-par_81.srm

Submitted batch job 788113
Submitted batch job 788114
Submitted batch job 788115
Submitted batch job 788116
Submitted batch job 788117
Submitted batch job 788118
Submitted batch job 788119


In [9]:
%%bash
# 1x1=1, 2x2=4, 3x3=9, 4x4=16, 6x6=36, 7x7=49, 8x8=64, 9x9=81
#sbatch st-nu-par_01.srm
sbatch st-nu-par_04.srm
#sbatch st-nu-par_09.srm
#sbatch st-nu-par_16.srm
#sbatch st-nu-par_36.srm
#sbatch st-nu-par_49.srm
#sbatch st-nu-par_64.srm
#sbatch st-nu-par_81.srm

Submitted batch job 788314


In [7]:
! squeue -n stnupar

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
            788106 cpu_small  stnupar xxxx. PD       0:00      1 (Resources)
            788107 cpu_small  stnupar xxxx. PD       0:00      1 (Resources)
            788108 cpu_small  stnupar xxxx. PD       0:00      1 (Resources)
            788109 cpu_small  stnupar xxxx. PD       0:00      2 (Resources)
            788110 cpu_small  stnupar xxxx. PD       0:00      3 (Resources)
            788111 cpu_small  stnupar xxxx. PD       0:00      3 (Resources)
            788112 cpu_small  stnupar xxxx. PD       0:00      4 (Resources)
            788113 cpu_small  stnupar xxxx. PD       0:00      1 (Resources)
            788114 cpu_small  stnupar xxxx. PD       0:00      1 (Resources)
            788115 cpu_small  stnupar xxxx. PD       0:00      1 (Resources)
            788116 cpu_small  stnupar xxxx. PD       0:00      2 (Resources)
            788117 cpu_small  stnupar xxxx. PD       0:00      3 (Re

In [11]:
! squeue -n stnupar

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)


In [12]:
%%bash
d='/scratch/ampemi/xxxx.xxxx/stnc/Numba'
cat $d/slurm-788113.out  #01
cat $d/slurm-788314.out  #04
cat $d/slurm-788114.out  #09
cat $d/slurm-788115.out  #16
cat $d/slurm-788116.out  #36
cat $d/slurm-788117.out  #49
cat $d/slurm-788118.out  #64
cat $d/slurm-788119.out  #81

- Job ID: 788113
- Tarefas por no:
- Qtd. de nos: 1
- Tot. de tarefas: 1
- Nos alocados: sdumont1149
sdumont1149
- diretorio onde sbatch foi chamado ($SLURM_SUBMIT_DIR):
/prj/ampemi/xxxx.xxxx/stnc/Numba
-- srun -------------------------------
$ srun --mpi=pmi2 -n 1 python st-nu-par.py
Heat=1500.0000 | Tempo=30.5408 | MPISize=1 | Dim=1,1 | bx,by=4800,4800
-- FIM --------------------------------
- Job ID: 788314
- Tarefas por no:
- Qtd. de nos: 1
- Tot. de tarefas: 4
- Nos alocados: sdumont1083
sdumont1083
- diretorio onde sbatch foi chamado ($SLURM_SUBMIT_DIR):
/prj/ampemi/xxxx.xxxx/stnc/Numba
-- srun -------------------------------
$ srun --mpi=pmi2 -n 4 python st-nu-par.py
Heat=1500.0000 | Tempo=8.4508 | MPISize=4 | Dim=2,2 | bx,by=2400,2400
-- FIM --------------------------------
- Job ID: 788114
- Tarefas por no:
- Qtd. de nos: 1
- Tot. de tarefas: 9
- Nos alocados: sdumont1149
sdumont1149
- diretorio onde sbatch foi chamado ($SLURM_SUBMIT_DIR):
/prj/ampemi/xxxx.xxxx/stnc/Numba
-- s